In [ ]:
import requests as rqs
import base64
from PIL import Image
import numpy as np
import math
from bs4 import BeautifulSoup as soup 
import threading     
import time 
print("Готово !")

В интернете существует множество сервисов, предоставляющих списки бесплатных прокси-серверов. Увы, все они **хотят с нас денег**. Бесплатный доступ тоже есть. Но он либо ограничен не слишком большим суточным лимитом, если речь идёт об удобном получении прокси через API, например в формате json. Либо приняты специальные меры, чтобы максимально затруднить сканирование и автоматическое получение списка прокси со страничек сайта. Вот так делаются деньги на бесплатном ресурсе... Подробно эту тему я не копал, но нашёл неплохой сервис ProxyRotator. Нет никаких хитрых проверок на "человечность" клиента. Чтобы затруднить сканирование, поле ip-адреса замусоривается невидимыми элементами, а номер порта выводится в виде картинки в формате png. Отсеять невидимые элементы и распознать число на картинке не составляет особого труда. Немного волшебства, и как говорится в древнем анекдоте, **польский офицер с русской пани денег не берёт :)))** Цена всего этого колдунства - полторы сотни строк на питоне. Давайте напишем функцию, получающую список прокси с ProxyRotator.


In [ ]:
# Считывает картинку в виде массива Numpy
def loadImgNp(file):
    img=Image.open(file)                #грузим изображение из файла
    w, h=img.size                       #w, h - его ширина и высота
    rr=[]                               #массив пикселов изображения
    for y in range(0, h):               #проход по вертикали
        xx=[]                           #массив пикселов горизонтальной строки изображения 
        for x in range(0, w):           #проход по горизонтали
            r,g,b,a=img.getpixel((x,y)) #получаем пиксел в виде компонент r, g, b, a
            xx.append(r+g+b+a)          #записываем сумму компонент
        rr.append(xx)                   #помещаем в массив строку пикселов
    a=np.array(rr, dtype=np.int)        #преобразуем в массив numpy
    return a-a.min()                    #вычитаем уровень фона

# Загружает предварительно подготовленные картинки цифр
def loadDigits():
    res=[]
    for i in range(0, 10):
        d=loadImgNp("./images/a"+str(i)+".png")  #грузим картинку
        d=d-(d.sum()/d.size)                     #вычитаем средний уровень
        res.append(d)                            #добавляем в массив
    return res
                
# Находит номер порта
def getRotatorPort(td):
    img=str(td.find("img").attrs["src"]) #находим атрибут src тега img
    pos=img.find(",")                    #находим в найденной строке подстроку после запятой  
    img=img[pos+1:]
    bin=base64.b64decode(img)            #декодируем её как base64    
    file=open("./port.png", "wb")        #записываем результат в бинарный файл port.png
    file.write(bin)
    file.close()
    return scanImage("./port.png")       #сканируем картинку, опознавая число

# Сканирует картинку и распознает число. digits - массив где хранятся картинки цифр
# Алгоритм тупейший и примитивнейший. Но 100% точности распознавания нам и не надо.
# Мусора в списке и так полно. Если иногда будут ошибки, оно не сильно повредит.
def scanImage(file):
    img=loadImgNp(file)                   #загружаем картинку из файла в numpy-массив   
    HH, WW=img.shape                      #HH, WW - её высота и ширина 
    d=img.sum(axis=0)                     #суммируем массив вдоль вертикальной оси 
    pos=0                                 #текущая позиция по горизонтали
    num=0                                 #текущее значение числа
    for pos in range(0, d.size):          #пропускаем пустое место в начале картинки
        if d[pos]>0:
            break
    while (WW-pos)>=10:                   #сканируем до тех пор, пока по горизонтали останется меньше 10 пикселов
        d=[]                              #массив оценок сходства для каждой из 10 цифр
        w=[]                              #массив смещений для каждой из 10 цифр
        for i in range(0, len(digits)):   #в текущей позиции проверяем сходство с каждой из 10 цифр
            s=[]                          #массив оценок сходства для трёх попыток со смещениями
            hh, ww=digits[i].shape        #высота и ширина текущей картинки цифры
            for j in range(0, 3):         #делаем три попытки, смещаясь каждый раз на 1 пиксель
                dd=img[:,pos+j:pos+j+ww]  #вырезаем из картинки кусок с текущей позиции плюс смещение j, равный по ширине текущей цифре
                dd=dd-(dd.sum()/dd.size)  #вычитаем средний уровень
                u=(dd*digits[i]).sum()    #вычисляем оценку сходства - скалярное произведение куска и текущей цифры
                s.append(u)               #помещаем её в массив попыток 
            mx=max(s)                     #находим наибольшую оценку среди попыток
            w.append(s.index(mx)+ww)      #помещаем соответствующее ей смещение в массив смещений
            d.append(mx)                  #а саму оценку в массив оценок цифр
        idx=d.index(max(d))               #найденная текущая цифра - цифра с наибольшей оценкой
        num=num*10+idx                    #добавляем эту цифру к числу
        pos+=w[idx]                       #а текущую позицию увеличиваем на соответствующее смещение
    return num
    
# Находит сколько секунд назад прокси был проверен    
def getRotatorAgo(td):
    a=str(td.contents[0]).split(" ")   #разбиваем строку по пробелу  
    t=int(a[0])                        #первая часть содержит число
    s=a[1][0]                          #вторая секунды, минуты или часы
    if s=="m":                         #смотрим что у нас и домножаем на 60 или 3600
        t*=60
    if s=="h":
        t*=3600
    return t

# Находит ip-адрес прокси
def getRotatorIP(nones, td):
    res=""
    a=td.contents
    for i in range(0, len(a)):        #сканируем все элементы
        elem=a[i]
        if str(elem)[0]=="<":
            if len(elem.contents)>0:  #если это нормальный тег с непустым содержимым
                cts=elem.contents[0]  #достаем содержимое
                s=str(elem)
                f=True
                for nn in nones:      #смотрим, входит ли в элемент какой-то класс
                    p=s.find(nn)      #означающий невидимость
                    if p>=0:
                        f=False
                        break
                if f:                 #если невидимости не обнаружено, 
                    res+=cts          #добавляем содержимое к результирующей строке
    return res

# Находит скорость
def getRotatorSpeed(td):
    a=td.find_all("i")                #смотрим, сколько в содержимом встретится
    n=0                               #элементов с классом "fa-star-o"
    for i in range(0, len(a)):        #это пустые звездочки
        s=str(a[i])                   #Скорость равна длинне тега минус число пустых звездочек
        p=s.find("fa-star-o")
        if p>=0:
            n+=1
    return len(a)-n   

# Находит классы, определённые как невидимые
def getRotatorNones(tr):
    nones=["none"]                     #В начале тега tr расположен тег style 
    style=tr.find("style")             #содержащий классы, некоторые из которых 
    s=str(style.contents[0])           #включают display:none
    a=s.split("}")                     #находим и собираем имена таких классов
    for s in a:
        p=s.find("none")
        if p>0:
            p=s.find(".")
            p1=s.find("{")
            nones.append(s[p+1:p1])
    return nones        

# Сканирует одну страничку. page - 1...10 - номер странички
def scanRotatorPage(page):
    res=[]
    url="https://www.proxyrotator.com/free-proxy-list/"+str(page)+"/"
    attempts=0
    while attempts<10:
        try:
            rsp=rqs.get(url, timeout=5)    #С 10 попыток соединяемся с сервером
            break
        except:
            attempts+=1
    if attempts==10:
        return res
    s=str(rsp.content.decode(rsp.encoding))  
    
    #========#     Проверочный код     #========#
    file=open("./test.html", "wt")
    file.write(s)
    file.close()
    #========# Конец проверочного кода #========#
    
    bsObj=soup(s, "lxml")  
    tbody=bsObj.find("tbody")            #Находим содержимое таблицы - тег tbody
    trs=tbody.find_all("tr")             #Находим все строки таблицы - tr
    for tr in trs:                       #Сканируем их всех
        proxy={}
        tds=tr.find_all("td")
        nones=getRotatorNones(tr)
        proxy["ago"]=getRotatorAgo(tds[0])    
        proxy["ip"]=getRotatorIP(nones, tds[1])
        proxy["port"]=getRotatorPort(tds[2])
        proxy["location"]=tds[3].contents[1].strip()
        proxy["speed"]=getRotatorSpeed(tds[4])
        proxy["protocol"]=str(tds[5].contents[0]).strip()
        proxy["anonimous"]=str(tds[6].contents[0]).strip()
        res.append(proxy)
    return res  

print("Готово !")

Проверить как это работает, непосредственно сравнивая результат с картинкой в броузере, нельзя. Список прокси на ресурсе меняется очень быстро. И запросы от нашего сканера и броузера скорее всего отобразят различные списки прокси. Поэтому в функцию scanRotatorPage() добавлен проверочный код, сохраняющий резальтат запроса в файле test.html. Если затем этот файл открыть в броузере, он будет содержать те же прокси, что и в полученном списке. В боевом варианте функции этот код можно удалить.

In [ ]:
# Выполните код и убедитесь, что список серверов на сохранённой html-страничке
# такой же как в распечатке.
digits=loadDigits()
proxies=scanRotatorPage(1)
for proxy in proxies:
    print("ip="+proxy["ip"]+", port="+str(proxy["port"])+", speed="+str(proxy["speed"]))    

Проверим, как наши прокси  работают. Для этого в сети есть ресурс icanhazip, который просто показывает ip-адрес, с которого на него зашли.Кликните по ссылке и увидите свой ip. Заходя через прокси мы естественно увидим другой ip-адрес.

In [ ]:
urlIpTest="http://icanhazip.com/"
s=""
try:
    rsp=rqs.get(urlIpTest)
    s=str(rsp.content.decode(rsp.encoding))
except:
    s="Ошибка"
print("Наш ip-адрес  ---> "+s)

timeout=5    #Зададим таймаут 5 секунд, чтобы не ждать слишком медленных прокси
             #Вы эксперимента ради, можете задать его и больше. Меньше уже нежелательно
             #Слишком много серверов отвалятся с ошибкой
n=0
nn=0
speed=0     #Здесь Вы можете задать отбор прокси по скорости, от 1 до 5  
for proxy in proxies:
    if proxy["speed"]>=speed:
        n+=1
        server=proxy["ip"]+":"+str(proxy["port"])
        prox={"http" : server}
        t1=time.time()
        try:
            rsp=rqs.get(urlIpTest, proxies=prox, timeout=timeout)
            s=str(rsp.content.decode(rsp.encoding))
            if(s[0]=="<"):
                s="Забанен"
            else:
                nn+=1
        except:
            s="Ошибка"
        t2=time.time()    
        t=str(math.floor(t2-t1))
        print(server+" v="+str(proxy["speed"])+" => "+s+" t="+t)
print("Выбрано прокси: "+str(n))
print("Без ошибок: "+str(nn))
    

По результатам теста видно, что сервис сыплет очень много мусора. Почему - не знаю. Возможно это специально делают в бесплатном доступе для любителей халявы. Однако где-то на треть работающих прокси мы вполне можем рассчитывать. Учитывая что число запросов у нас неограничено, нет никаких суточных лимитов, это очень и очень неплохо. Давайте теперь напишем функцию, которая пытается получить число прокси не меньше заданного, со скоростью не меньше заданной и всех их проверяет на сайте избиркома. Чтобы это работало быстро, будем запускать проверку на множестве потоков.

In [ ]:
#Пытается получить не менее чем num прокси, со скоростью не менее speed,
#и таймаутом соединения timeout
def getProxyList(num, speed, timeout):
    # Внимание !!! Я хочу сканировать сайт избиркома. Поэтому проверяю по его URL
    # Вы должны подставить URL того сайта, с которым хотите соединяться через прокси
    url="http://www.vybory.izbirkom.ru/region/izbirkom?action=show&global=1&vrn=100100031793505&region=0&prver=0&pronetvd=null"
    res=[]                       #результат
    nth=0                        #число потоков
    lock=threading.Lock()        #замок
    done=threading.Event()       #событие окончания всех проверок
    
    #Вложенная функция пытающаяся соединиться с сервером избиркома
    #и в случае получения какого-то ответа проверить, а тот ли это ответ что нужен
    def test(p):                
        nonlocal url, res, nth, lock, done
        f=False
        proxy={"http":p["ip"]+":"+str(p["port"])} 
        #Соединяемся и проверяем содержимое странички
        #В случае успеха устанавливаем флаг f в True
        try: 
            rsp=rqs.get(url, proxies=proxy, timeout=timeout)
            s=str(rsp.content.decode(rsp.encoding))
            bsObj=soup(s, "lxml") 
            #Здесь я проверяю содержимое странички
            # Вы должны в этом месте вставить свою проверку
            select=bsObj.find("select")
            opt=select.find_all("option")[12] 
            if str(opt.contents[0]).strip()=="Республика Марий Эл":
                f=True
        except:
            f=False
        lock.acquire()     #Запираем замок, тут должно быть только однопоточное исполнение
        if nth>0:          #Уменьшаем число потоков
            nth-=1
        if f:              #Если проверили успешно, добавляем прокси к списку
            res.append(p)
        if nth==0:         #Если число потоков равно нулю, запускаем событие
            done.set()
        lock.release()     #освобождаем замок
                    
    for i in range(0, 10):             #сканируем страницы с 1 по 10       
        proxy=scanRotatorPage(i+1)
        for p in proxy:
            if p["speed"]>=speed:      #если прокси подошел по скорости
                nth+=1                 #запускаем его проверку в новом потоке
                th=threading.Thread(target=test, args=[p])
                th.start()
        done.wait()                    #ждём завершения всех проверок
        if len(res)>num:               #если собрали достаточно прокси, выходим
            break
    return res


In [ ]:
# Проверяем
digits=loadDigits()            # Загружаем цифры
lst=getProxyList(10, 3, 5)     # Пытаемся найти не менее 10 прокси, со скоростью не менне 3 и таймаутом 5 секунд
print(len(lst))                # Печатаем длинну списка
for proxy in lst:              # Печатаем весь список
    print(proxy)